# Synchronous Generator DCIM vs VBR model

In [ ]:
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import matplotlib.pyplot as plt
import re
import numpy as np
import math
import os
import subprocess

#%matplotlib widget
timestep = 10e-6
duration = 1.0

dpsim_path = subprocess.Popen(['git', 'rev-parse', '--show-toplevel'], stdout=subprocess.PIPE).communicate()[0].rstrip().decode('utf-8')
path_exec = dpsim_path + '/build/dpsim/examples/cxx/'

## DCIM simulation

In [ ]:
model_name = 'EMT_SynGenDQ7odTrapez_SMIB_Fault'

sim = subprocess.Popen([path_exec+model_name, '--name', model_name, '--timestep', str(timestep), '--duration', str(duration)], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
print(sim.communicate()[0].decode())

path_res = 'logs/' + model_name + '/'
dpsim_result_file_res = path_res  + model_name + '.csv'
ts_dcim = read_timeseries_csv(dpsim_result_file_res)

## VBR simulation

In [ ]:
model_name = 'EMT_SynGenVBR_SMIB_Fault'

sim = subprocess.Popen([path_exec+model_name, '--name', model_name, '--timestep', str(timestep), '--duration', str(duration)], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
print(sim.communicate()[0].decode())

path_res = 'logs/' + model_name + '/'
dpsim_result_file_res = path_res  + model_name + '.csv'
ts_vbr = read_timeseries_csv(dpsim_result_file_res)

## Definition ROI

In [ ]:
t_begin=0
t_end=1

begin_idx = int(t_begin/timestep)
end_idx= int(t_end/timestep)

## Network node voltages

In [ ]:
plt.figure(figsize=(12,8))
plt.ylabel('Network node voltage (V)', fontsize=18)

for name in ['v1', 'v2']:
    plt.plot(ts_dcim[name + '_0'].time[begin_idx:end_idx], np.sqrt(3/2)*ts_dcim[name + '_0'].values[begin_idx:end_idx], label=name+' (DCIM)')
    plt.plot(ts_vbr[name + '_0'].time[begin_idx:end_idx], np.sqrt(3/2)*ts_vbr[name + '_0'].values[begin_idx:end_idx], label=name+' (VBR)', linestyle='--')
    
plt.legend(fontsize=14)
plt.show()

## Generator terminal voltage

In [ ]:
plt.figure(figsize=(12,8))
plt.ylabel('Generator terminal voltage (V)', fontsize=18)

for name in ['v_gen']:
    plt.plot(ts_dcim[name + '_0'].time[begin_idx:end_idx], np.sqrt(3/2)*ts_dcim[name + '_0'].values[begin_idx:end_idx], label=name+' (DCIM)')
    plt.plot(ts_vbr[name + '_0'].time[begin_idx:end_idx], np.sqrt(3/2)*ts_vbr[name + '_0'].values[begin_idx:end_idx], label=name+' (VBR)', linestyle='--')
    
plt.legend(fontsize=14)
plt.show()

## Genrerator terminal Current

In [ ]:
plt.figure(figsize=(12,8))
plt.ylabel('Generator terminal current (A)', fontsize=18)

for name in ['i_gen']:
    plt.plot(ts_dcim[name + '_0'].time[begin_idx:end_idx], np.sqrt(3/2)*ts_dcim[name + '_0'].values[begin_idx:end_idx], label=name+' (DCIM)')
    plt.plot(ts_vbr[name + '_0'].time[begin_idx:end_idx], np.sqrt(3/2)*ts_vbr[name + '_0'].values[begin_idx:end_idx], label=name+' (VBR)', linestyle='--')

plt.legend()
plt.show()

## Voltage across line

In [ ]:
plt.figure(figsize=(12,8))

for name in ['v_line']:
    plt.plot(ts_dcim[name + '_0'].time[begin_idx:end_idx], np.sqrt(3/2)*ts_dcim[name + '_0'].values[begin_idx:end_idx], label=name+' (DCIM)')
    plt.plot(ts_vbr[name + '_0'].time[begin_idx:end_idx], np.sqrt(3/2)*ts_vbr[name + '_0'].values[begin_idx:end_idx], label=name+' (VBR)', linestyle='--')

plt.legend()
plt.show()

## Current through line

In [ ]:
plt.figure(figsize=(12,8))

for name in ['i_line']:
    plt.plot(ts_dcim[name + '_0'].time[begin_idx:end_idx], np.sqrt(3/2)*ts_dcim[name + '_0'].values[begin_idx:end_idx], label=name+' (DCIM)')
    plt.plot(ts_vbr[name + '_0'].time[begin_idx:end_idx], np.sqrt(3/2)*ts_vbr[name + '_0'].values[begin_idx:end_idx], label=name+' (VBR)', linestyle='--')

plt.legend()
plt.show()

## Rotor frequency  $f_r$

In [ ]:
plt.figure(figsize=(12,8))
plt.xlabel('time (s)', fontsize=20)
plt.ylabel('Rotor frequency (Hz)', fontsize=20)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)

for name in ['wr_gen']:
    plt.plot(ts_dcim[name].time[begin_idx:end_idx], ts_dcim[name].values[begin_idx:end_idx]*60, label=name+' (DCIM)')
    plt.plot(ts_vbr[name].time[begin_idx:end_idx], ts_vbr[name].values[begin_idx:end_idx]*60, label=name+' (VBR)', linestyle='--')

plt.legend(fontsize=18)
plt.show()

## Rotor angular velocity $\omega _r$

In [ ]:
plt.figure(figsize=(12,8))
plt.ylabel('Rotor angular velocity (rad/s)', fontsize=18)


for name in ['wr_gen']:
    plt.plot(ts_dcim[name].time[begin_idx:end_idx], 2*math.pi*60*ts_dcim[name].values[begin_idx:end_idx], label=name+' (DCIM)')
    plt.plot(ts_vbr[name].time[begin_idx:end_idx], 2*math.pi*60*ts_vbr[name].values[begin_idx:end_idx], label=name+' (VBR)', linestyle='--')

plt.legend()
plt.show()

## Rotor angle $\delta _r$

In [ ]:
plt.figure(figsize=(12,8))
for name in ['delta_r']:
    plt.plot(ts_dcim[name].time[begin_idx:end_idx], ts_dcim[name].values[begin_idx:end_idx]*180/3.14, label=name+' (DCIM)')
    plt.plot(ts_vbr[name].time[begin_idx:end_idx], ts_vbr[name].values[begin_idx:end_idx]*180/3.14, label=name+' (VBR)', linestyle='--')
plt.legend()
plt.show()

## Machine Torques

In [ ]:
plt.figure(figsize=(12,8))
for name in ['T_e']:
    plt.plot(ts_dcim[name].time[begin_idx:end_idx], ts_dcim[name].values[begin_idx:end_idx], label=name+' (DCIM)')
    plt.plot(ts_vbr[name].time[begin_idx:end_idx], -ts_vbr[name].values[begin_idx:end_idx], label=name+' (VBR)', linestyle='--')
for name in ['T_m']:
    plt.plot(ts_dcim[name].time[begin_idx:end_idx], ts_dcim[name].values[begin_idx:end_idx], label=name+' (DCIM)')
    plt.plot(ts_vbr[name].time[begin_idx:end_idx], -ts_vbr[name].values[begin_idx:end_idx], label=name+' (VBR)', linestyle='--')
plt.legend()
plt.show()

## Assertion

In [ ]:
diff = {}
for name in ['v_gen_0', 'v_gen_1', 'v_gen_2']:
    diff[name] = ts_dcim[name].rmse(ts_dcim[name], ts_vbr[name])/np.max(ts_dcim[name].values)
    print(name + ': ' + str(diff[name]))
    assert(diff[name]) < 1.86e-6
for name in ['i_gen_0', 'i_gen_1', 'i_gen_2']:
    diff[name] = ts_dcim[name].rmse(ts_dcim[name], ts_vbr[name])/np.max(ts_dcim[name].values)
    print(name + ': ' + str(diff[name]))
    assert(diff[name]) < 5e-4